# Handle multiple LLM models in Langchain and AWS Bedrock seamlessly

This notebook example shows how to integrate langchain with mulitple LLM models and corresponding prompt with AWS Bedrock.

### Create a multi LLM langchain interface

In [41]:
from langchain_community.chat_models import BedrockChat
from langchain_core.runnables import ConfigurableField

In [42]:
aws_region_name = "us-west-2"
credentials_profile_name = "default"
claude_3_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
mistral_large = "mistral.mistral-large-2402-v1:0"

In [43]:
mistral_large_bedrock_chat = BedrockChat(
    model_id=mistral_large,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)

In [44]:
_model_alternatives = {
    "mistral_large": mistral_large_bedrock_chat
}

In [45]:
claude_3_sonnet = BedrockChat(
    model_id=claude_3_sonnet,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)

In [46]:
bedrock_llm = claude_3_sonnet.configurable_alternatives(
    which=ConfigurableField(
        id="model", name="Model", description="The model that will be used"
    ),
    default_key="claude_3_sonnet",
    **_model_alternatives,
)

In [47]:
bedrock_llm.invoke("who are you ?")

AIMessage(content='I am an AI assistant created by Anthropic.  I am an artificial intelligence without a physical body or form.  My purpose is to help humans like yourself with a variety of tasks like analysis, writing, research, answering questions, and more.', response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 11, 'completion_tokens': 55, 'total_tokens': 66}}, id='run-6d6e9cad-8808-4e07-81ca-74bcb1dfa7c6-0')

In [48]:
bedrock_llm \
    .with_config(configurable={"model": "mistral_large"}) \
    .invoke("who are you ?")

AIMessage(content=" I am a language model trained by the Mistral AI team. I am designed to understand and generate human-like text based on the input I receive. I'm here to provide information, answer questions, and engage in conversation on a wide range of topics.", response_metadata={'model_id': 'mistral.mistral-large-2402-v1:0', 'usage': {'prompt_tokens': 8, 'completion_tokens': 54, 'total_tokens': 62}}, id='run-2fd96810-63c9-43be-8489-77170768fcf2-0')

### Create a multi LLM langchain prompt

In [49]:
from langchain_core.prompts import PromptTemplate

In [50]:
_MISTRAL_PROMPT = PromptTemplate.from_template(
    """
<s>[INST] You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in rhymes.

Human:
<human_reply>
{input}
</human_reply>

Generate the AI's response.[/INST]</s>
"""
)

In [51]:
_CLAUDE_PROMPT = PromptTemplate.from_template(
    """
You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in jokes.

Human:
<human_reply>
{input}
</human_reply>

Assistant:
"""
)

In [52]:
_CHAT_PROMPT_ALTERNATIVES = {"mistral_large": _MISTRAL_PROMPT}

In [53]:
CONFIGURABLE_CHAT_PROMPT = _CLAUDE_PROMPT.configurable_alternatives(
    which=ConfigurableField(
        id="model",
        name="Model",
        description="The model that will be used",
    ),
    default_key="claude_3_sonnet",
    **_CHAT_PROMPT_ALTERNATIVES
)

### Create the multi LLM langchain chain

In [54]:
from langchain.schema.output_parser import StrOutputParser

In [55]:
chain = CONFIGURABLE_CHAT_PROMPT | bedrock_llm | StrOutputParser()

In [56]:
chain.invoke(input="What is a large language model ?")

"*chuckles* A large language model? Well, it's kind of like a really talkative parrot that's been reading a lot of books! It's an AI that can spew out sentences and paragraphs like nobody's business. Though, to be fair, I probably shouldn't be calling it names - that would be a bit bird-brained of me! *laughs*"

In [57]:
bedrock_llm \
    .with_config(configurable={"model": "mistral_large"}) \
    .invoke("What is a large language model ?")

AIMessage(content=' A large language model is a type of artificial intelligence model that has been trained on a vast amount of text data. It\'s designed to generate human-like text based on the input it receives. This can range from answering questions, writing essays, summarizing texts, translating languages, and even generating creative content like poetry or stories.\n\nThe "large" in large language model refers to the size of the model in terms of the number of parameters it has. These models can have billions or even trillions of parameters, which are the parts of the model that are learned from the training data. The more parameters a model has, the more it can learn about the complexities and nuances of language.\n\nHowever, it\'s important to note that while large language models can generate human-like text, they don\'t actually understand the text in the way humans do. They\'re simply predicting what word or phrase should come next based on patterns they\'ve learned from the